# Lab #1 Images, optics, and the statistics of light

In [1]:
# Import necessary packages
import numpy as np
import math

## Optical Imager
*GOAL*: Map an object to an image plane (image formation). \
\
**Thin-lens equation**:

$$
\frac{1}{f} = \frac{1}{i} + \frac{1}{o}
$$

where $f = \text{focal length of thin lens}$, $o = \text{distance from object to thin lens}$ and $i = \text{distance from image to thin lens}$. 

**Focal length** = determines how strongly a system converges or diverges recieved light, determines the extent of an image and the distance between two imaged objects on the focal plane. \
**Focal plane** = where your image comes to a focus \
**Combination of two lenses equation:** 
$$
\frac{1}{F} = \frac{1}{f_1} + \frac{1}{f_2} - \frac{d}{f_1 * f_2}
$$

where $F = \text{combined focal length}$, $f_1 = \text{focal length of lens 1}$, $f_2 = \text{focal length of lens 2}$ and $d = \text{distance between lenses}$.

In [2]:
# Detector
detector_width_pixels = 1440 # pixels
detector_height_pixels = 1440 # pixels
pixel_size_um = 2.4 # um

In [3]:
# Lenses
focal_lengths_mm = [50.0, 75.0, 100.0]

In [4]:
# Optical bench setup
distance_from_detector_to_object_mm = 330 # mm
min_distance_between_two_lenses_mm = 5 #mm
max_distance_between_two_lenses_mm = 200 #mm

In [5]:
# Ronchi Mask
num_ronchi_lines_from_75_mm_lens = 17

## 1.1 Plate Scale
*GOAL*: Determine an optimal imaging system with a plate scale that would allow to resolve two successive lines of the provided Ronchi mask. \
\
**Plate scale** = angular size of an object that can be imaged onto a particular linear size on the focal plane (angle / unit length) (radians / mm) (arcseconds / pixel)


$$
\begin{split}
\text{Plate Scale } (p) & = \frac{\text{angular separation}\,(\text{radians})}{\text{linear separation} (\text{mm})} = \frac{\text{pixel size (mm)}}{\text{focal length (mm)}} = \frac{\text{radians}}{\text{pixel}} * \left(\frac{206265\,\text{arcseconds}}{1\,\text{radian}} \right) = \frac{\text{arcseconds}}{\text{pixel}}
\end{split}
$$

In [6]:
def calculate_plate_scale_and_field_of_view(pixel_size_um,
                                            focal_length_mm,
                                            detector_width_pixels,
                                            detector_height_pixels):
    # Plate scale
    arcseconds_in_a_radian = 206265
    focal_length_um = focal_length_mm * 1000
    plate_scale = arcseconds_in_a_radian * (pixel_size_um / focal_length_um)

    # Field of View
    field_of_view_width_arcseconds = plate_scale * detector_width_pixels
    field_of_view_height_arcseconds = plate_scale * detector_height_pixels
                                                
    return plate_scale, field_of_view_width_arcseconds, field_of_view_height_arcseconds

for focal_length_mm in focal_lengths_mm:
    plate_scale, field_of_view_width_arcseconds, field_of_view_height_arcseconds = calculate_plate_scale_and_field_of_view(pixel_size_um=pixel_size_um,
                                                                                                                           focal_length_mm=focal_length_mm,
                                                                                                                           detector_width_pixels=detector_width_pixels,
                                                                                                                           detector_height_pixels=detector_height_pixels)
    
    print(f"Plate scale for {focal_length_mm}mm lens is {np.round(plate_scale, 4)} (arcseconds / pixel).")
    print(f"Field of view for {focal_length_mm}mm lens is {np.round(field_of_view_width_arcseconds, 4)} x {np.round(field_of_view_height_arcseconds, 4)}  (arcseconds).\n")
    

Plate scale for 50.0mm lens is 9.9007 (arcseconds / pixel).
Field of view for 50.0mm lens is 14257.0368 x 14257.0368  (arcseconds).

Plate scale for 75.0mm lens is 6.6005 (arcseconds / pixel).
Field of view for 75.0mm lens is 9504.6912 x 9504.6912  (arcseconds).

Plate scale for 100.0mm lens is 4.9504 (arcseconds / pixel).
Field of view for 100.0mm lens is 7128.5184 x 7128.5184  (arcseconds).



In [9]:
calculate_plate_scale_and_field_of_view(pixel_size_um=2.4,
                                            focal_length_mm=32,
                                            detector_width_pixels=1440,
                                            detector_height_pixels=1440)

(15.469874999999998, 22276.62, 22276.62)

In [12]:
# Calculate ronchi line separatation
def calculate_ronchi_line_separation_mm(num_ronchi_lines, detector_width_pixels, focal_length_mm, pixel_size_um):
    pixel_size_mm = pixel_size_um / 1000 # mm
    detector_width_mm = detector_width_pixels * pixel_size_mm

    line_separation_mm = detector_width_mm / (num_ronchi_lines - 1) # Subtract by 1 since there are N-1 spaces between N lines

    return line_separation_mm

# Pre-emptively determine the distance between Ronchi lines
ronchi_line_separation_mm = calculate_ronchi_line_separation_mm(num_ronchi_lines=num_ronchi_lines_from_75_mm_lens,
                                                                detector_width_pixels=detector_width_pixels, 
                                                                focal_length_mm=focal_lengths_mm[1], 
                                                                pixel_size_um=pixel_size_um)

print(f"Separation between Ronchi lines: {np.round(ronchi_line_separation_mm,4)} (mm).")

Separation between Ronchi lines: 0.216 (mm).


In [13]:
def calculate_size_in_arcseconds(separation_on_image_mm, distance_to_image_mm):
    theta_radians = math.atan(separation_on_image_mm / distance_to_image_mm) #radians
    theta_degrees = math.degrees(theta_radians) # degrees 
    theta_arcseconds = theta_degrees * 3600 # arcseconds
    return theta_arcseconds

ronchi_line_separation_arcseconds = calculate_size_in_arcseconds(separation_on_image_mm=ronchi_line_separation_mm,
                                                                 distance_to_image_mm=distance_from_detector_to_object_mm)

print(f"Separation between Ronchi lines is {np.round(ronchi_line_separation_arcseconds,4)} (arcseconds).")

Separation between Ronchi lines is 135.0097 (arcseconds).


## 1.2 Field of View
Determine an optimal imaging system that is capable of imaging the entire extent of Jupiter in a single exposure.

In [14]:
diameter_of_jupiter_mm = 110 # mm

In [15]:
diameter_of_jupiter_arcseconds = calculate_size_in_arcseconds(separation_on_image_mm=diameter_of_jupiter_mm,
                                                              distance_to_image_mm=1000)

print(f"Diameter of Jupiter is {np.round(diameter_of_jupiter_arcseconds,4)} (arcseconds).")

Diameter of Jupiter is 22598.2746 (arcseconds).


In [13]:
def calculate_needed_plate_scale(size_in_arcseconds, number_of_pixels):
    plate_scale = size_in_arcseconds / number_of_pixels
    return plate_scale

jupiter_needed_plate_scale = calculate_needed_plate_scale(size_in_arcseconds=diameter_of_jupiter_arcseconds,
                                                          number_of_pixels=detector_height_pixels)

print(f"To image Jupiter, the plate scale needs to be {np.round(jupiter_needed_plate_scale, 4)} (arcseconds / pixel).")

To image Jupiter, the plate scale needs to be 32.4052 (arcseconds / pixel).


In [42]:
def calculate_plate_scale_to_focal_length_mm(plate_scale, pixel_size_um):
    pixel_size_mm = pixel_size_um / 1000
    focal_length = (206265 * pixel_size_mm) / plate_scale
    return focal_length 

jupiter_needed_focal_length = calculate_plate_scale_to_focal_length_mm(plate_scale=jupiter_needed_plate_scale, pixel_size_um=pixel_size_um)

print(f"To image Jupiter, the focal length needs to be {np.round(jupiter_needed_focal_length, 4)}mm. ")

To image Jupiter, the focal length needs to be 15.2764mm. 


In [44]:
def combined_focal_length(lens_1_focal_length_mm, lens_2_focal_length_mm, distance_between_lenses):
    step1 = (1 / lens_1_focal_length_mm) +  (1 / lens_2_focal_length_mm)
    step2 = (distance_between_lenses / (lens_1_focal_length_mm * lens_1_focal_length_mm))
    print(step2)
    step3 = 1 / (step1 - step2)
    
    return step3

combined_min_focal_length_50_75 = combined_focal_length(lens_1_focal_length_mm=focal_lengths_mm[0],
                                                        lens_2_focal_length_mm=focal_lengths_mm[1],
                                                        distance_between_lenses=min_distance_between_two_lenses_mm)

combined_max_focal_length_50_75 = combined_focal_length(lens_1_focal_length_mm=focal_lengths_mm[0],
                                                        lens_2_focal_length_mm=focal_lengths_mm[1],
                                                        distance_between_lenses=max_distance_between_two_lenses_mm)

combined_min_focal_length_75_100 = combined_focal_length(lens_1_focal_length_mm=focal_lengths_mm[1],
                                                        lens_2_focal_length_mm=focal_lengths_mm[2],
                                                        distance_between_lenses=min_distance_between_two_lenses_mm)

combined_max_focal_length_75_100 = combined_focal_length(lens_1_focal_length_mm=focal_lengths_mm[1],
                                                         lens_2_focal_length_mm=focal_lengths_mm[2],
                                                         distance_between_lenses=max_distance_between_two_lenses_mm)

combined_min_focal_length_50_100 = combined_focal_length(lens_1_focal_length_mm=focal_lengths_mm[0],
                                                        lens_2_focal_length_mm=focal_lengths_mm[2],
                                                        distance_between_lenses=min_distance_between_two_lenses_mm)

combined_max_focal_length_50_100 = combined_focal_length(lens_1_focal_length_mm=focal_lengths_mm[0],
                                                         lens_2_focal_length_mm=focal_lengths_mm[2],
                                                         distance_between_lenses=max_distance_between_two_lenses_mm)

print(f"Combined focal length of {focal_lengths_mm[0]}mm lens and {focal_lengths_mm[1]}mm lens at {min_distance_between_two_lenses_mm}mm apart is {np.round(combined_min_focal_length_50_75, 4)}mm.")
print(f"Combined focal length of {focal_lengths_mm[0]}mm lens and {focal_lengths_mm[1]}mm lens at {max_distance_between_two_lenses_mm}mm apart is {np.round(combined_max_focal_length_50_75, 4)}mm.\n")

print(f"Combined focal length of {focal_lengths_mm[0]}mm lens and {focal_lengths_mm[2]}mm lens at {min_distance_between_two_lenses_mm}mm apart is {np.round(combined_min_focal_length_50_100, 4)}mm.")
print(f"Combined focal length of {focal_lengths_mm[0]}mm lens and {focal_lengths_mm[2]}mm lens at {max_distance_between_two_lenses_mm}mm apart is {np.round(combined_max_focal_length_50_100, 4)}mm.\n")

print(f"Combined focal length of {focal_lengths_mm[1]}mm lens and {focal_lengths_mm[2]}mm lens at {min_distance_between_two_lenses_mm}mm apart is {np.round(combined_min_focal_length_75_100, 4)}mm.")
print(f"Combined focal length of {focal_lengths_mm[1]}mm lens and {focal_lengths_mm[2]}mm lens at {max_distance_between_two_lenses_mm}mm apart is {np.round(combined_max_focal_length_75_100, 4)}mm.")

0.002
0.08
0.0008888888888888889
0.035555555555555556
0.002
0.08
Combined focal length of 50.0mm lens and 75.0mm lens at 5mm apart is 31.9149mm.
Combined focal length of 50.0mm lens and 75.0mm lens at 200mm apart is -21.4286mm.

Combined focal length of 50.0mm lens and 100.0mm lens at 5mm apart is 35.7143mm.
Combined focal length of 50.0mm lens and 100.0mm lens at 200mm apart is -20.0mm.

Combined focal length of 75.0mm lens and 100.0mm lens at 5mm apart is 44.5545mm.
Combined focal length of 75.0mm lens and 100.0mm lens at 200mm apart is -81.8182mm.


In [41]:
def calculate_needed_distance(combined_focal_length_mm, focal_length_1_mm, focal_length_2_mm):
    step1 = 1 / combined_focal_length_mm
    step2 = (1 / focal_length_1_mm) +  (1 / focal_length_2_mm) 
    step3 = step2 - step1
    step4 = step3 * (focal_length_1_mm * focal_length_2_mm)
    return step4

calculated_distance_between_lenses_50_75 = calculate_needed_distance(combined_focal_length_mm=jupiter_needed_focal_length,
                                                                     focal_length_1_mm=focal_lengths_mm[0],
                                                                     focal_length_2_mm=focal_lengths_mm[2])

calculated_distance_between_lenses_50_100 = calculate_needed_distance(combined_focal_length_mm=jupiter_needed_focal_length,
                                                                      focal_length_1_mm=focal_lengths_mm[0],
                                                                      focal_length_2_mm=focal_lengths_mm[1])

calculated_distance_between_lenses_75_100 = calculate_needed_distance(combined_focal_length_mm=jupiter_needed_focal_length,
                                                                      focal_length_1_mm=focal_lengths_mm[1],
                                                                      focal_length_2_mm=focal_lengths_mm[2])

print(f"To image Jupiter, the distance between {focal_lengths_mm[0]}mm lens and {focal_lengths_mm[2]}mm lens must be {np.round(calculated_distance_between_lenses_50_75, 4)}.")
print(f"To image Jupiter, the distance between {focal_lengths_mm[0]}mm lens and {focal_lengths_mm[1]}mm lens must be {np.round(calculated_distance_between_lenses_50_100, 4)}.")
print(f"To image Jupiter, the distance between {focal_lengths_mm[1]}mm lens and {focal_lengths_mm[2]}mm lens must be {np.round(calculated_distance_between_lenses_75_100, 4)}.")

To image Jupiter, the distance between 50.0mm lens and 100.0mm lens must be -177.3013.
To image Jupiter, the distance between 50.0mm lens and 75.0mm lens must be -120.476.
To image Jupiter, the distance between 75.0mm lens and 100.0mm lens must be -315.9519.


## Field of View & Plate Scale
What is the plate scale achieved with your optical system that imaged the entire extent of Jupiter? What plate scale should be used to resolve Jupiter spots and smallest strucutres? \
\
Use the mock globular cluster to determine the minimal separation between "stars" in order to resolve them with your previous optical system to estimate pixel size. 

## Detector Properties
